# 8.1 ODE II

#### Before we start:
* revisit 7.2 convergence test

#### Today's class:

* 3D line plots
* ODE's with mulitple coupled equations
    - _Lotka–Volterra_ equations and comparison of two solvers
    - Discuss and understand the accuracy of a numerical solution, and how to use libraries properly
    - Conclusions and recommendations
* Chaos: Lorenz equations

    

## 3D line plots
Just in case we have to solve systems with three equations it may be useful to remind us of how to do 3D line plots.

In [ ]:
%pylab ipympl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# plot 1
x = lambda t: t*cos(t)
y = lambda t: 1.5*t*sin(t)
z = lambda t: t**2
t = linspace(0,50,10001)

In [ ]:
# plot 2
def x(t):
    t_ = fac * t
    return t_*cos(t_)
def y(t):
    t_ = fac * t
    return 1.5*t_*sin(t_)
# z = lambda t: t**2 * t**-0.3
z = lambda x: sin(t**2)*x**2/exp(-t/2.) # from Lab 6.1
t = linspace(-2.5,0.25,10001); fac = 100

In [ ]:
ifig=19;close(ifig);fig = plt.figure(ifig)
ax = fig.gca(projection='3d',azim=-30, elev=45)
ax.plot(x(t),y(t),z(t))
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
show()

## ODE's with mulitple coupled equations
The [Lotka–Volterra equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations), also known as the predator–prey equations, represent a model for an interacting predator and a prey population:
 
 $$ \frac{dx}{dt} = \alpha x - \beta x y$$ 
 
 $$\frac{dy}{dt} = \delta xy -\gamma y$$ 
 
 where:
 * $x$ number of prey
 * $y$ number of predetors
 * and $\alpha$, $\beta$, $\delta$ and $\gamma$ are parameters that describe the interaction between the two species.
 
1. Write the code to solve the Lotka–Volterra equations equations using first the `scipy.integrate.odeint` and  then the `integrate.solve_ivp` method. 
2. Create a plot with the evolution of prey and predetor for the parameters $\alpha=2/3$, $\beta=4/3$, $\delta=1$ and $\gamma=1/2$ for the time interval $[0,35]$ for each solution. Make a plot that shows the prey population which shows both solutions. Are you satisfied with the agreement?
3. Integrate the ODE with both solvers to $t=350$ and plot the prey population. Also zoom in to view in detail just the last 35 time units. Are you still satisfied? What is the solver you prefer for this problem, and why?

Note: make sure that you are using for each line a different compination of linestyle and color to make distinguishing the lines very easy.

In [ ]:
%pylab ipympl
from scipy.integrate import odeint

In [ ]:
# parameters and initial conditions
a=4/3; b=4/3; d=1.0; g=1/2
state0 = [0.2, 0.5]
tmin,tmax = (0,350)

### `odeint` solution

* Before we start using the `odeint` let's explore what is in it! 
* Look at and find [documentation](https://computing.llnl.gov/casc/nsde/pubs/u113855.pdf)

In [ ]:
# odeint?

In [ ]:
def fl(state, t):
    x, y = state  # unpack the state vector
    return a*x - b*x*y, d*x*y - g*y

t = arange(tmin, tmax, 0.01)

states = odeint(fl, state0, t)



In [ ]:
ifig = 7; close(ifig); figure(ifig)
plot(t,states[:,0],label='odeint, prey')
plot(t,states[:,1],'-.',label='odeint, predator')
legend(loc=0); xlabel('time'); ylabel('population')

### `solve_ivp` solution

As it turns out there is one caveat with `integrate.solve_ivp`, which is how it deals with arguments to the RHS. It does not have a built-in way to do this. 

In [ ]:
from scipy.integrate import solve_ivp
def fl_sivp(t,state,parms):
    a,b,d,g = parms  # unpack parameter vector
    x, y = state     # unpack the state vector
    return (a*x - b*x*y, d*x*y - g*y)

state0 = array([0.2, 0.5])
parms=(a,b,d,g)

Another point to note is that this library will automatically pick the time step size. 

In [ ]:
t_eval = arange(tmin, tmax, 0.01)

In [ ]:
rhs = lambda t,y: fl_sivp(t,y,parms)
sol = solve_ivp(rhs,[tmin,tmax],state0,t_eval=t_eval)
#  default: 1e-3 for `rtol` and 1e-6 for `atol`

In [ ]:
# inspect solution
sol

In [ ]:
ifig = 3; close(ifig); figure(ifig)
plot(t_eval,sol.y[0],label='prey')
plot(t_eval,sol.y[1],label='predator')
legend(loc=0); xlabel('time'); ylabel('population')
title('solve_ivp')

### Observation
The solution fluctuates with this solver in an irregular pattern. Is this correct?

In [ ]:
close(8);figure(8)
plot(t_eval,sol.y[0],'--',label='solve_ivp, prey')
plot(t,states[:,0],label='odeint, prey')
legend()


The comparison between the two solvers shows this to be a substantial effect. Could the `solve_ivp` be incorrect? Is the predetor-prey model maybe chaotic? No! [The Poincaré–Bendixson theorem states that a two-dimensional differential equation has very regular behavior.](https://en.wikipedia.org/wiki/Chaos_theory). The `solve_ivp` solver with the default methods and parameters simply does not do a very good job for this problem. This becomes clear when we compare the evolution of prey at the end of a longer run. Not only the amplitude differs by >10%.  The phase has shifted as well.

In [ ]:
close(9);figure(9)
plot(t,states[:,0],label='odeint, prey')
plot(t_eval,sol.y[0],'--',label='solve_ivp, prey')
legend()
xlim(0.9*tmax,tmax)

**What is going on?**

In [ ]:
# solve_ivp?

### Conclusions and recommendations
* library can have a number of solver behaviours
* properties of solvers may be optional arguments, such as tolerance
* read the documentation

## Lorenz equations
One of the most celebrated sets of differential equations in physics is the [Lorenz equations](https://en.wikipedia.org/wiki/Lorenz_system):
$$ \frac{dx}{dt} = \sigma(y-x) $$ $$\frac{dy}{dt} = rx -y -xz$$ $$\frac{dz}{dt} = xy - bz$$ where $\sigma$, $r$ and $b$ are constants. (The names of these constants may seem arbirtrary and odd but are always used in these equations -- _for historical reasons_).

Use the range from $t = 0$ to $t = 50 $ with the intial condition $(x,y,z) = (1,1,1)$, and solve with parameters $\sigma=10.0$, $r=28$ and $b=8/3$.

Use the scipy solver `integrate.solve_ivp`.

In [ ]:
%pylab ipympl
from scipy import integrate

In [ ]:
def lorenz_rhs_ode(t,yy,params):
    '''
    Righ-hand-side (RHS) of Lorenz equations for scipy.integrate.solve_ivp
    
    Parameters:
    -----------
    t : float
      time 
    params : tuple, floats
      (s,r,b) sigma, r, b parameters
    yy : array, float
      position vector, three components

    Return: 
    -------
    rhs : list, floats 
      RHS new position vector
    '''

    s,r,b = params
    x,y,z = yy
    rhs = array([ s*(y-x), r*x -y -x*z, x*y - b*z])
    return rhs

In [ ]:
# set the parameters for this solution
s = 7.0; r = 32; b = 9./3
params = (s,r,b)
tmin,tmax = (0,25)

nmax = 2000
t_eval = linspace(tmin,tmax,nmax)
fun = lambda t,y : lorenz_rhs_ode(t,y,params)
a0  = array([1.,1.,1.])
sol    = integrate.solve_ivp(fun,[tmin,tmax],a0, t_eval = t_eval,rtol=1.e-6 )

#### Time evolution of the first component

In [ ]:
close(1);figure(1)
plot(sol.t,sol.y[0],label='x')
plot(sol.t,sol.y[1],label='y')
plot(sol.t,sol.y[2],label='z')
xlabel('time'); ylabel("x coordinate")
legend()

In [ ]:
close(11);figure(11)
plot(sol.t,sol.y[0],label='x')
plot(sol.t,sol.y[1],label='y')
plot(sol.t,sol.y[2],label='z')
xlabel('time'); ylabel("x coordinate")
legend()

#### Plot of x vs y component

In [ ]:
cm = plt.cm.get_cmap('coolwarm')
n = linspace(1,nmax+1,nmax)
color = cm(n/nmax)[::-1]

In [ ]:
close(2);figure(2)
for i in range(nmax-2):
    plot(sol.y[0][i:i+2],sol.y[1][i:i+2],'-',c=color[i],lw=0.75)
plot(sol.y[0][-2],sol.y[1][-2],'o',c=color[-2])
xlabel('x'); ylabel("y")

In [ ]:
close(12);figure(12)
for i in range(nmax-2):
    plot(sol.y[0][i:i+2],sol.y[1][i:i+2],'-',c=color[i],lw=0.75)
plot(sol.y[0][-2],sol.y[1][-2],'o',c=color[-2])
xlabel('x'); ylabel("y")

In [ ]:
close(3);figure(3)
for i in range(nmax-2):
    plot(sol.y[0][i:i+2],sol.y[2][i:i+2],'-',c=color[i],lw=0.75)
plot(sol.y[0][-2],sol.y[2][-2],'o',c=color[-2])
xlabel('x'); ylabel("z")

In [ ]:
close(13);figure(13)
for i in range(nmax-2):
    plot(sol.y[0][i:i+2],sol.y[2][i:i+2],'-',c=color[i],lw=0.75)
plot(sol.y[0][-2],sol.y[2][-2],'o',c=color[-2])
xlabel('x'); ylabel("z")

#### Last but not least, let's do a 3D plot of the 3D trajectory.

In [ ]:
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
ifig=4;close(ifig);fig = plt.figure(ifig)
ax = fig.gca(projection='3d',azim=-30, elev=45)
for i in range(nmax-2):
    ax.plot(sol.y[0][i:i+2],sol.y[1][i:i+2],sol.y[2][i:i+2],'-',c=color[i],lw=0.75)
ax.plot(sol.y[0][-2:-1],sol.y[1][-2:-1],sol.y[2][-2:-1],'o',c=color[-2])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
show()

In [ ]:
ifig=14;close(ifig);fig = plt.figure(ifig)
ax = fig.gca(projection='3d',azim=-30, elev=45)
for i in range(nmax-2):
    ax.plot(sol.y[0][i:i+2],sol.y[1][i:i+2],sol.y[2][i:i+2],'-',c=color[i],lw=0.75)
ax.plot(sol.y[0][-2:-1],sol.y[1][-2:-1],sol.y[2][-2:-1],'o',c=color[-2])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
show()